In [1]:
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG
from ray.tune.logger import pretty_print
import ray
import json
import pandas as pd
import gym

In [2]:
ray.shutdown()
ray.init()

2021-10-16 15:33:01,182	INFO services.py:1252 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '172.16.19.112',
 'raylet_ip_address': '172.16.19.112',
 'redis_address': '172.16.19.112:6379',
 'object_store_address': '/tmp/ray/session_2021-10-16_15-32-59_366008_35876/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-10-16_15-32-59_366008_35876/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-10-16_15-32-59_366008_35876',
 'metrics_export_port': 64484,
 'node_id': '3994c905d1decccc0833115ae26f91f3d6221fadb4cd11e953b89c7f'}

In [3]:
config = DEFAULT_CONFIG.copy()
config["num_workers"] = 1
config["num_sgd_iter"] = 30
config["sgd_minibatch_size"] = 128
config["model"]["fcnet_hiddens"] = [100, 100]
config["num_cpus_per_worker"] = 1

agent = PPOTrainer(config, "CartPole-v1")

2021-10-16 15:33:06,779	INFO trainer.py:741 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-10-16 15:33:06,780	INFO ppo.py:165 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2021-10-16 15:33:06,781	INFO trainer.py:760 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=36311) WARNING:tensorflow:From /home/ec2-user/anaconda3/envs/tensorflow_p36/gpu_cuda10.0/lib/python3.6/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
(pid=36311) 
(pid=36311) 2021-10-16 15:33:11.379409: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(pid=36311) WARNING:tensorflow:From /home/ec2-user/anaconda3/envs/tensorflow

(RolloutWorker pid=36311) [2021-10-16 15:33:16.162 ip-172-16-19-112:36311 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
(RolloutWorker pid=36311) [2021-10-16 15:33:16.308 ip-172-16-19-112:36311 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[2021-10-16 15:34:18.720 ip-172-16-19-112:35876 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-10-16 15:34:18.754 ip-172-16-19-112:35876 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


2021-10-16 15:34:19,297	WARNING trainer_template.py:186 -- `execution_plan` functions should accept `trainer`, `workers`, and `config` as args!
2021-10-16 15:34:19,297	INFO trainable.py:112 -- Trainable.setup took 72.519 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2021-10-16 15:34:19,298	WARNING util.py:57 -- Install gputil for GPU system monitoring.


In [12]:
from ray import tune

def objective(step, alpha, beta):
    return (0.1 + alpha * step / 100)**(-1) + beta * 0.1


def training_function(config):
    # Hyperparameters
    alpha, beta = config["alpha"], config["beta"]
    for step in range(30):
        # Iterative training function - can be any arbitrary training procedure.
        intermediate_score = objective(step, alpha, beta)
        # Feed the score back back to Tune.
        tune.report(mean_loss=intermediate_score)

analysis = tune.run(
    training_function,
    config={
        "alpha": tune.grid_search([0.001, 0.01, 0.1]),
        "beta": tune.choice([1, 2, 3])
    })

print("Best config: ", analysis.get_best_config(
    metric="mean_loss", mode="min"))

# Get a dataframe for analyzing trial results.
df = analysis.results_df
df
"""
analysis = ray.tune.run(
    ppo.PPOTrainer,
    config=config,
    local_dir=log_dir,
    stop=stop_criteria,
    checkpoint_at_end=True)
"""
"""
alg = 'PPO'
tune.run(alg,
    stop={'episode_reward_mean':200},
    config={
        'env':'CartPole-v0',
        'num_gpus':1,
        'num_workers':2,
        'lr':tune.grid_search([.01,.001,.0001])     
    }
)
"""

2021-10-16 21:24:39,633	WARNING tune.py:562 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,alpha,beta
training_function_787aa_00000,PENDING,,0.001,3
training_function_787aa_00001,PENDING,,0.01,2
training_function_787aa_00002,PENDING,,0.1,1


Result for training_function_787aa_00000:
  date: 2021-10-16_21-24-40
  done: false
  experiment_id: 6a058826c651496a84ffdf111bceb2fb
  hostname: ip-172-16-19-112
  iterations_since_restore: 1
  mean_loss: 10.3
  neg_mean_loss: -10.3
  node_ip: 172.16.19.112
  pid: 36266
  time_since_restore: 0.00018477439880371094
  time_this_iter_s: 0.00018477439880371094
  time_total_s: 0.00018477439880371094
  timestamp: 1634419480
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 787aa_00000
  
Result for training_function_787aa_00001:
  date: 2021-10-16_21-24-40
  done: false
  experiment_id: 875e44dbc30f45188b8950114ea6da53
  hostname: ip-172-16-19-112
  iterations_since_restore: 1
  mean_loss: 10.2
  neg_mean_loss: -10.2
  node_ip: 172.16.19.112
  pid: 36264
  time_since_restore: 0.000885009765625
  time_this_iter_s: 0.000885009765625
  time_total_s: 0.000885009765625
  timestamp: 1634419480
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 787aa_00001
  
Result fo

Trial name,status,loc,alpha,beta,loss,iter,total time (s),neg_mean_loss
training_function_787aa_00000,TERMINATED,,0.001,3,10.291,10,0.110491,-10.291
training_function_787aa_00001,TERMINATED,,0.01,2,10.1108,10,0.12166,-10.1108
training_function_787aa_00002,TERMINATED,,0.1,1,9.27431,10,0.0916603,-9.27431


2021-10-16 21:24:40,712	INFO tune.py:617 -- Total run time: 1.09 seconds (0.96 seconds for the tuning loop).


Best config:  {'alpha': 0.1, 'beta': 1}


,mean_loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,neg_mean_loss,experiment_id,date,timestamp,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,experiment_tag,config.alpha,config.beta
trial_id,,,,,,,,,,,,,,,,,,,,
787aa_00000,10.291008,0.016557,True,None,None,10,-10.291008,6a058826c651496a84ffdf111bceb2fb,2021-10-16_21-24-40,1634419480,0.110491,36266,ip-172-16-19-112,172.16.19.112,0.110491,0,10,"0_alpha=0.001,beta=3",0.001,3
787aa_00001,10.110803,0.020331,True,None,None,10,-10.110803,875e44dbc30f45188b8950114ea6da53,2021-10-16_21-24-40,1634419480,0.121660,36264,ip-172-16-19-112,172.16.19.112,0.121660,0,10,"1_alpha=0.01,beta=2",0.010,2
787aa_00002,9.274312,0.008116,True,None,None,10,-9.274312,4d147de793704de2a0b4654d3547e6e8,2021-10-16_21-24-40,1634419480,0.091660,36258,ip-172-16-19-112,172.16.19.112,0.091660,0,10,"2_alpha=0.1,beta=1",0.100,1


In [7]:
def objective(x, a, b):
    return a * (x ** 0.5) + b

def trainable(config):
    # config (dict): A dict of hyperparameters.

    for x in range(20):
        intermediate_score = objective(x, config["a"], config["b"])

        tune.report(score=intermediate_score)  # This sends the score to Tune.

analysis = tune.run(
    trainable,
    config={"a": 2, "b": 4}
)

print("best config: ", analysis.get_best_config(metric="score", mode="max"))

2021-10-16 21:16:22,345	WARNING tune.py:562 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc
trainable_50126_00000,PENDING,


Result for trainable_50126_00000:
  date: 2021-10-16_21-16-23
  done: false
  experiment_id: 0e5f14345bc14735b33fed2cdc048f63
  hostname: ip-172-16-19-112
  iterations_since_restore: 1
  node_ip: 172.16.19.112
  pid: 36282
  score: 4.0
  time_since_restore: 0.0001614093780517578
  time_this_iter_s: 0.0001614093780517578
  time_total_s: 0.0001614093780517578
  timestamp: 1634418983
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '50126_00000'
  
Result for trainable_50126_00000:
  date: 2021-10-16_21-16-23
  done: true
  experiment_id: 0e5f14345bc14735b33fed2cdc048f63
  experiment_tag: '0'
  hostname: ip-172-16-19-112
  iterations_since_restore: 20
  node_ip: 172.16.19.112
  pid: 36282
  score: 12.717797887081348
  time_since_restore: 0.06075167655944824
  time_this_iter_s: 0.0030181407928466797
  time_total_s: 0.06075167655944824
  timestamp: 1634418983
  timesteps_since_restore: 0
  training_iteration: 20
  trial_id: '50126_00000'
  


Trial name,status,loc,iter,total time (s),score
trainable_50126_00000,TERMINATED,,20,0.0607517,12.7178


2021-10-16 21:16:23,332	INFO tune.py:617 -- Total run time: 0.99 seconds (0.88 seconds for the tuning loop).


best config:  {'a': 2, 'b': 4}
